#Import Data From Keggle and Unzip That

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d salader/dogs-vs-cats

100% 1.06G/1.06G [00:10<00:00, 85.2MB/s]
100% 1.06G/1.06G [00:10<00:00, 108MB/s] 


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
!mkdir -p data/train
!mkdir -p data/valid

#iamge aug

In [5]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

In [6]:
batch_size = 16

#this is the augumentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale= 1./255,
    shear_range=0.2, #dishort <<--    -->>
    zoom_range=0.2, #zoom_in - out
    horizontal_flip=True)


#this is the augumentation configuration we will use for testing:
#only rescaling
test_datagen = ImageDataGenerator(rescale= 1./255)

#Generator that read picture found in subfolder of 'data/train'
#batches of augumeted image data
train_generator = train_datagen.flow_from_directory(
    '/content/train', #this is target directory
    target_size=(150,150), #all images will be resize into 150 x 150
    batch_size = batch_size,
    class_mode='binary' # since we use binary_crossentropy loss, we need binary labels
)

#this is similar generator for validation data
validation_generator = train_datagen.flow_from_directory(
    '/content/test', #this is target directory
    target_size=(150,150), #all images will be resize into 150 x 150
    batch_size = batch_size,
    class_mode='binary' # since we use binary_crossentropy loss, we need binary labels
)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [10]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, models, regularizers
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), padding='valid', activation='relu', input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Conv2D(64, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Conv2D(128, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Conv2D(256, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=2, padding='valid'))

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 74, 74, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 36, 36, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 17, 17, 128)      

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [13]:
model.fit_generator(
    train_generator, #steps_per_epoch=2000, #//batch_size
    epochs=5,
    validation_data=validation_generator,
    #validation_steps = 800, #// batch_size

)

Epoch 1/5


<ipython-input-13-9d22e2d7cfb8>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


1250/1250 [==============================] - 181s 143ms/step - loss: 0.6854 - accuracy: 0.5462 - val_loss: 0.6708 - val_accuracy: 0.5756
Epoch 2/5
1250/1250 [==============================] - 185s 148ms/step - loss: 0.6336 - accuracy: 0.6384 - val_loss: 0.6325 - val_accuracy: 0.6330
Epoch 3/5
1250/1250 [==============================] - 177s 141ms/step - loss: 0.5446 - accuracy: 0.7266 - val_loss: 0.4981 - val_accuracy: 0.7650
Epoch 4/5
1250/1250 [==============================] - 176s 141ms/step - loss: 0.4698 - accuracy: 0.7746 - val_loss: 0.4378 - val_accuracy: 0.7968
Epoch 5/5
1250/1250 [==============================] - 175s 140ms/step - loss: 0.4217 - accuracy: 0.8051 - val_loss: 0.3991 - val_accuracy: 0.8170
